In [1]:
import pandas as pd

In [2]:
train_df = pd.read_csv('./基于论文摘要的文本分类与查询性问答公开数据/train.csv', sep=',')
test_df = pd.read_csv('./基于论文摘要的文本分类与查询性问答公开数据/test.csv', sep=',')

In [3]:
train_df.head()

,Title,Authors,Citation,Abstract,DOI,Topic(Label)
0,\n \n \n \n \n \n \n The Va...,"['Seyhmus Tunc', 'Suleyman Cemil Oglak', 'Fatm...",2022 Jun;32(6):722-727.,\n\n\n Objective:\n \n \n...,\n doi: 10.29271/jcpsp.2022.06.722.\n ...,Abdominal+Fat
1,\n \n \n \n \n \n \n Metfor...,"['Katrin Schmitz', 'Eva-Maria Turnwald', 'Tobi...",2022 May 30;14(11):2288.,\n\n \n With the gaining prevalence ...,\n doi: 10.3390/nu14112288.\n,Abdominal+Fat
2,\n \n \n \n \n \n \n New In...,"['Jorge Valencia-Ortega', 'Rebeca GonzÃ¡lez-Re...",2022 Jun 3;23(11):6279.,\n\n \n Gestational diabetes mellitu...,\n doi: 10.3390/ijms23116279.\n,Abdominal+Fat
3,\n \n \n \n \n \n \n Î±-Lin...,"['Victoria Aiassa', 'MarÃ­a Del Rosario Ferrei...",2022 Jun;156:111164.,\n\n \n Given obesity and its associ...,\n doi: 10.1016/j.foodres.2022.111164.\...,Abdominal+Fat
4,\n \n \n \n \n \n \n Perire...,"['Ming Jiang', 'Menghuan Li', 'Cuiying Liu', '...",2022 May 6;13:865009.,\n\n\n Background:\n \n \...,\n doi: 10.3389/fendo.2022.865009.\n,Abdominal+Fat


In [4]:
test_df.head()

,Title,Authors,Citation,Abstract,DOI
0,\n \n \n \n \n \n \n Gut mi...,"['Xiaomin Su', 'Minying Zhang', 'Houbao Qi', '...",2022 Jan 24;10(1):13.,\n\n\n Background:\n \n \...,\n doi: 10.1186/s40168-021-01205-8.\n ...
1,\n \n \n \n \n \n \n [A spa...,"['Naigong Yu', 'Yishen Liao', 'Naigong Yu', 'Y...",2022 Apr 25;39(2):217-227.,\n\n \n Physiological studies reveal...,\n doi: 10.7507/1001-5515.202109051.\n ...
2,\n \n \n \n \n \n \n A muci...,"['Ju-Hyung Lee', 'Soo-Jeong Kwon', 'Ji-Yoon Ha...",2022 Feb;60(2):215-223.,\n\n \n The mammalian intestinal tra...,\n doi: 10.1007/s12275-022-1649-3.\n
3,\n \n \n \n \n \n \n Progre...,"['Michal Å tros', 'Eva VolfovÃ¡ PolanskÃ¡', 'T...",2022 Apr 5;12(4):544.,\n\n \n Extracellular HMGB1 protein ...,\n doi: 10.3390/biom12040544.\n
4,\n \n \n \n \n \n \n Microb...,"['Jing Ouyang', 'Silvere D Zaongo', 'Xue Zhang...",2022 Jan 6;12:755890.,\n\n \n Hepatitis B virus (HBV) co-i...,\n doi: 10.3389/fimmu.2021.755890.\n


In [5]:
train_df['Topic(Label)'], lbl = pd.factorize(train_df['Topic(Label)'])

In [6]:
train_df['Title'] = train_df['Title'].apply(lambda x: x.strip())
train_df['Abstract'] = train_df['Abstract'].fillna('').apply(lambda x: x.strip())
train_df['text'] = train_df['Title'] + ' ' + train_df['Abstract']
train_df['text'] = train_df['text'].str.lower()

test_df['Title'] = test_df['Title'].apply(lambda x: x.strip())
test_df['Abstract'] = test_df['Abstract'].fillna('').apply(lambda x: x.strip())
test_df['text'] = test_df['Title'] + ' ' + test_df['Abstract']
test_df['text'] = test_df['text'].str.lower()

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score

tfidf = TfidfVectorizer(max_features=2500)

In [8]:
train_tfidf = tfidf.fit_transform(train_df['text'])

clf = SGDClassifier()
cross_val_score(clf, train_tfidf, train_df['Topic(Label)'], cv=5)

/home/lyz/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


array([0.88821569, 0.88245986, 0.88882157, 0.89215389, 0.8769697 ])

In [9]:
test_tfidf = tfidf.transform(test_df['text'])

clf = SGDClassifier()
clf.fit(train_tfidf, train_df['Topic(Label)'])
test_df['Topic(Label)'] = clf.predict(test_tfidf)

In [10]:
test_df['Topic(Label)'] = test_df['Topic(Label)'].apply(lambda x: lbl[x])
test_df[['Topic(Label)']].to_csv('submit.csv', index=None)